##Importing all required libraries

In [0]:
import pandas as pd
from keras import Sequential
from keras import layers
from keras import regularizers
import numpy as np
from string import punctuation
from collections import Counter
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Using TensorFlow backend.


##Spearman Score Calculation Method 

We pass two parameter to the method 
model predicted value and actual value to calculate Spearman Score

In [0]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

## KFold Cross-Validation API

In [0]:
# scikit-learn k-fold cross-validation
from numpy import array
from sklearn.model_selection import KFold

def kfold_train_test_split(train,test, ):
  # prepare cross validation
  kfold = KFold(10, True, 1)
  # enumerate splits
  for train, test in kfold.split(data):
    print('train: %s, test: %s' % (data[train], data[test]))


##Connecting google drive to notebook

Few places I have loaded features of all videos in a single csv file for the ease of loading it into dataframe and performing predictions

In [0]:
from google.colab import drive
import os
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
ground_truth_path = '/content/drive/My Drive/CA684_Assignment_Data/test-set/dev-set_ground-truth.csv'
groundTruth=pd.read_csv(ground_truth_path)

In [0]:
groundTruth.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


#Caption

##Starting with captions of video as per literature review captions provide the most significant correlation

In [0]:
def read_caps(fname):
    """Load the captions into a dataframe"""
    vn = []
    cap = []
    df = pd.DataFrame();
    with open(fname) as f:
        for line in f:
            pairs = line.split()
            vn.append(pairs[0])
            cap.append(pairs[1])
        df['video']=vn
        df['caption']=cap
    return df


In [0]:
captions_path = '/content/drive/My Drive/CA684_Assignment_Data/dev-set/dev-set_video-captions.txt'
captions=read_caps(captions_path)

In [62]:
captions.head(5)

,video,caption
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


In [0]:
dataset1 = pd.merge(groundTruth,captions, on ="video")

In [0]:
dataset1.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,caption
0,video3.webm,0.924,34,0.846,13,blonde-woman-is-massaged-tilt-down
1,video4.webm,0.923,33,0.667,12,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,0.863,33,0.700,10,khr-gangsters
3,video8.webm,0.922,33,0.818,11,medical-helicopter-hovers-at-airport
4,video10.webm,0.950,34,0.900,10,couple-relaxing-on-picnic-crane-shot


###Feature Engineering based on caption column

1.   Removing special character
2.   converting entire caption to lower case
3.   tokenize the array
4.   stemming using Porter Stemmer
5.   Removing English stopwords
6.   joining back to form clean corpus






In [0]:
import nltk
import re
import string
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words('english'))
snow = nltk.stem.LancasterStemmer('english')
ps = PorterStemmer() 
tokenizer = RegexpTokenizer(r'\w+')

corpus = [] 

for sentence in dataset1['caption'] :
  # column : "Review", row ith 
  review = re.sub('[^a-zA-Z]', ' ', sentence) 
  
  # convert all cases to lower cases 
  review = review.lower()  
  
  # split to array(default delimiter is " ") 
  review = tokenizer.tokenize(review)
 
  
  review = [ps.stem(word) for word in review if not word in stop_words] 
  
  # rejoin all string array elements 
    # to create back into a string 
  review = ' '.join(review)
  
  
 # append each string to create 
    # array of clean text  
  corpus.append(review) 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Adding it back to dataset1

In [0]:
dataset1['corpus']= corpus
dataset1.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,caption,corpus
0,video3.webm,0.924,34,0.846,13,blonde-woman-is-massaged-tilt-down,blond woman massag tilt
1,video4.webm,0.923,33,0.667,12,roulette-table-spinning-with-ball-in-closeup-shot,roulett tabl spin ball closeup shot
2,video6.webm,0.863,33,0.700,10,khr-gangsters,khr gangster
3,video8.webm,0.922,33,0.818,11,medical-helicopter-hovers-at-airport,medic helicopt hover airport
4,video10.webm,0.950,34,0.900,10,couple-relaxing-on-picnic-crane-shot,coupl relax picnic crane shot


###splitting test train for prediction

In [0]:
x_caption_short_train, x_caption_short_test, y_caption_short_train, y_caption_short_test = train_test_split(dataset1['corpus'], dataset1['short-term_memorability'], test_size=0.2, random_state=101)
x_caption_long_train, x_caption_long_test, y_caption_long_train, y_caption_long_test = train_test_split(dataset1['corpus'], dataset1['long-term_memorability'], test_size=0.2, random_state=101)

###Using TfidfVectorizer and ngram(unigram & bigram) to convert string to numeric values to feed to the model

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizerS = TfidfVectorizer(max_features=1500, ngram_range=(1,2) )
vectorizerS.fit(x_caption_short_train)

x_caption_short_train=vectorizerS.transform(x_caption_short_train)
x_caption_short_test=vectorizerS.transform(x_caption_short_test)





vectorizerL = TfidfVectorizer(max_features=1500, ngram_range=(1,2) )
vectorizerL.fit(x_caption_long_train)

x_caption_long_train=vectorizerS.transform(x_caption_long_train)
x_caption_long_test=vectorizerS.transform(x_caption_long_test)

###MODEL 1: Using LinearRegression 

In [0]:
from sklearn.linear_model import LinearRegression 

model_caption_short_linearReg = LinearRegression ()
model_caption_short_linearReg.fit(x_caption_short_train, y_caption_short_train)

output_caption_short_linearReg=model_caption_short_linearReg.predict(x_caption_short_test)

print('Short Term Memorability')
Get_score(output_caption_short_linearReg,y_caption_short_test)





model_caption_long_linearReg = LinearRegression ()
model_caption_long_linearReg.fit(x_caption_long_train, y_caption_long_train)

output_caption_long_linearReg=model_caption_long_linearReg.predict(x_caption_long_test)

print('Long Term Memorability')
Get_score(output_caption_long_linearReg,y_caption_long_test)

Short Term Memorability
The Spearman's correlation coefficient is: 0.356
Long Term Memorability
The Spearman's correlation coefficient is: 0.130


###MODEL 2: Using SVR

In [0]:
from sklearn.svm import SVR 

model_caption_short_svr = SVR (kernel='linear')
model_caption_short_svr.fit(x_caption_short_train, y_caption_short_train)

output_caption_short_svr=model_caption_short_svr.predict(x_caption_short_test)

print('Short Term Memorability')
Get_score(output_caption_short_svr,y_caption_short_test)






model_caption_long_svr = SVR (kernel='linear')
model_caption_long_svr.fit(x_caption_long_train, y_caption_long_train)

output_caption_long_svr=model_caption_long_svr.predict(x_caption_long_test)

print('Long Term Memorability')
Get_score(output_caption_long_svr,y_caption_long_test)

Short Term Memorability
The Spearman's correlation coefficient is: 0.303
Long Term Memorability
The Spearman's correlation coefficient is: 0.112


###MODEL 3: Using Bayesian Ridge

In [0]:
from sklearn.linear_model import BayesianRidge

model_caption_short_bayesian = BayesianRidge ()
model_caption_short_bayesian.fit(x_caption_short_train.toarray(), y_caption_short_train)

output_caption_short_bayesian=model_caption_short_bayesian.predict(x_caption_short_test.toarray())

print('Short Term Memorability')
Get_score(output_caption_short_bayesian,y_caption_short_test)






model_caption_long_bayesian = BayesianRidge ()
model_caption_long_bayesian.fit(x_caption_long_train.toarray(), y_caption_long_train)

output_caption_long_bayesian=model_caption_long_bayesian.predict(x_caption_long_test.toarray())

print('Long Term Memorability')
Get_score(output_caption_long_bayesian,y_caption_long_test)


Short Term Memorability
The Spearman's correlation coefficient is: 0.417
Long Term Memorability
The Spearman's correlation coefficient is: 0.154


#Aesthetic Features

In [0]:
aesthetic_path = '/content/drive/My Drive/dataframe_aesthetic.csv'
aesthetic=pd.read_csv(aesthetic_path,index_col=0)

In [0]:
aesthetic.head(5)

,0,1,2,3,4,5,6,7,8,9,...,100,101,102,103,104,105,106,107,108,video
0,0.581810,0.194872,0.531678,0.125891,0.480966,0.358678,52.473407,0.604938,0.000302,2.0,...,-0.000008,0.877948,0.280758,0.229451,1.477636,0.0,0.391471,0.060211,0.338636,video6633.webm
1,0.141382,0.218025,0.341507,0.124468,0.306108,0.481848,51.203502,0.166667,0.000389,3.0,...,0.000184,0.046312,0.050476,0.189169,0.487030,0.0,0.273047,0.151776,0.441400,video6637.webm
2,0.168417,0.089344,0.541439,0.153814,0.518419,0.240795,50.625188,0.125000,0.000553,2.0,...,-0.000000,0.000000,0.169706,0.044193,1.401191,0.0,0.299827,0.181719,0.402697,video6634.webm
3,0.604572,0.657199,0.465797,0.526192,0.323581,0.476609,56.950678,0.600000,0.025879,2.0,...,-0.000005,0.103966,0.046547,0.519998,1.098700,0.0,0.265116,0.340027,0.035195,video6635.webm
4,0.415735,0.455611,0.385227,0.316629,0.290064,0.473490,46.176749,0.585106,0.008992,4.0,...,-0.000009,0.996366,0.492007,0.418422,0.885331,0.0,0.144959,0.220941,0.589551,video6639.webm


In [0]:
dataset2 = pd.merge(groundTruth,aesthetic, on ="video")
dataset2.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,0,1,2,3,4,...,99,100,101,102,103,104,105,106,107,108
0,video3.webm,0.924,34,0.846,13,0.180874,0.715626,0.198930,0.610453,0.121610,...,-0.002154,-0.000850,0.000000,0.260845,0.294153,0.575061,0.0,0.129661,0.412381,0.788761
1,video4.webm,0.923,33,0.667,12,0.146076,0.550070,0.460729,0.424397,0.341807,...,0.000007,-0.000399,0.920270,0.337932,0.382134,0.943198,0.0,0.692224,0.762354,-0.561793
2,video6.webm,0.863,33,0.700,10,0.063898,0.827439,0.211925,0.720297,0.129499,...,0.002896,0.002309,0.374684,0.019155,0.187358,0.512601,0.0,0.438261,-0.219560,0.276687
3,video8.webm,0.922,33,0.818,11,0.380539,0.134683,0.746891,0.171604,0.697045,...,0.000568,0.000514,0.970656,0.435550,0.431595,1.142377,0.0,0.424483,0.326030,0.688481
4,video10.webm,0.950,34,0.900,10,0.427258,0.381448,0.582746,0.409475,0.489531,...,0.005307,0.000065,0.000000,0.442575,0.347355,1.285838,0.0,-0.279630,-0.012414,0.197281


###splitting test and train data set

In [0]:
x_aesthetic_short_train, x_aesthetic_short_test, y_aesthetic_short_train, y_aesthetic_short_test = train_test_split(dataset2.drop(['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations'], axis=1), dataset2['short-term_memorability'], test_size=0.2, random_state=101)
x_aesthetic_long_train, x_aesthetic_long_test, y_aesthetic_long_train, y_aesthetic_long_test = train_test_split(dataset2.drop(['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations'], axis=1), dataset2['long-term_memorability'], test_size=0.2, random_state=101)

###MODEL 1 : Linear Regression

In [0]:
from sklearn.linear_model import LinearRegression 

model_aesthetic_short_linearReg = LinearRegression ()
model_aesthetic_short_linearReg.fit(x_aesthetic_short_train, y_aesthetic_short_train)

output_aesthetic_short_linearReg=model_aesthetic_short_linearReg.predict(x_aesthetic_short_test)

print('Short Term Memorability')
Get_score(output_aesthetic_short_linearReg,y_aesthetic_short_test)





model_aesthetic_long_linearReg = LinearRegression ()
model_aesthetic_long_linearReg.fit(x_aesthetic_long_train, y_aesthetic_long_train)

output_aesthetic_long_linearReg=model_aesthetic_long_linearReg.predict(x_aesthetic_long_test)

print('Long Term Memorability')
Get_score(output_aesthetic_long_linearReg,y_aesthetic_long_test)

Short Term Memorability
The Spearman's correlation coefficient is: 0.237
Long Term Memorability
The Spearman's correlation coefficient is: 0.088


###MODEL 2 : DecisionTreeRegressor

In [0]:
from sklearn.tree import DecisionTreeRegressor

model_aesthetic_short_decisionTree = DecisionTreeRegressor ()
model_aesthetic_short_decisionTree.fit(x_aesthetic_short_train, y_aesthetic_short_train)

output_aesthetic_short_decisionTree=model_aesthetic_short_decisionTree.predict(x_aesthetic_short_test)

print('Short Term Memorability')
Get_score(output_aesthetic_short_decisionTree,y_aesthetic_short_test)





model_aesthetic_long_decisionTree = DecisionTreeRegressor ()
model_aesthetic_long_decisionTree.fit(x_aesthetic_long_train, y_aesthetic_long_train)

output_aesthetic_long_decisionTree=model_aesthetic_long_decisionTree.predict(x_aesthetic_long_test)

print('Long Term Memorability')
Get_score(output_aesthetic_long_decisionTree,y_aesthetic_long_test)


Short Term Memorability
The Spearman's correlation coefficient is: 0.150
Long Term Memorability
The Spearman's correlation coefficient is: 0.028


###MODEL 3 : Bayesian Ridge

In [0]:
from sklearn.linear_model import BayesianRidge

model_aesthetic_short_bayesian= BayesianRidge ()
model_aesthetic_short_bayesian.fit(x_aesthetic_short_train, y_aesthetic_short_train)

output_aesthetic_short_bayesian=model_aesthetic_short_bayesian.predict(x_aesthetic_short_test)

print('Short Term Memorability')
Get_score(output_aesthetic_short_bayesian,y_aesthetic_short_test)





model_aesthetic_long_bayesian = BayesianRidge ()
model_aesthetic_long_bayesian.fit(x_aesthetic_long_train, y_aesthetic_long_train)

output_aesthetic_long_bayesian=model_aesthetic_long_bayesian.predict(x_aesthetic_long_test)

print('Long Term Memorability')
Get_score(output_aesthetic_long_bayesian,y_aesthetic_long_test)



Short Term Memorability
The Spearman's correlation coefficient is: 0.248
Long Term Memorability
The Spearman's correlation coefficient is: 0.062


#C3D

In [0]:
c3d_path = '/content/drive/My Drive/c3d_data.csv'
c3d=pd.read_csv(c3d_path,index_col=0)

In [0]:
c3d.head(5)

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,video
0,0.010858,0.010386,0.000000,0.000000e+00,0.000000e+00,0.000000,2.700000e-07,0.000000,1.000000e-08,3.400000e-07,...,0.000000,0.000000,1.700000e-07,0.000000,0.000000e+00,1.000000e-08,1.300000e-06,2.600000e-06,8.000000e-08,video6632.webm
1,0.000200,0.000065,0.993807,2.000000e-07,4.700000e-07,0.000073,3.700000e-06,0.000337,6.710000e-06,2.290000e-06,...,0.000010,0.000002,5.800000e-06,0.000001,1.490000e-06,1.170000e-05,1.500000e-07,8.300000e-07,1.060000e-04,video6634.webm
2,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.120000e-06,video6633.webm
3,0.005959,0.004765,0.003757,5.787100e-04,7.344000e-05,0.000129,4.665300e-04,0.010266,3.028100e-04,2.196500e-04,...,0.003992,0.004137,3.670390e-02,0.000767,3.692100e-04,1.251980e-02,8.422000e-05,1.159825e-02,1.155040e-03,video6645.webm
4,0.005782,0.000306,0.004011,1.007000e-05,1.034000e-05,0.000002,3.160000e-06,0.000003,1.984000e-05,5.750000e-06,...,0.000002,0.000004,6.818350e-03,0.000005,5.000000e-08,2.088000e-05,1.271750e-03,4.862200e-04,1.965000e-05,video6643.webm


In [0]:
dataset3 = pd.merge(groundTruth,c3d, on ="video")
dataset3.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,0,1,2,3,4,...,91,92,93,94,95,96,97,98,99,100
0,video3.webm,0.924,34,0.846,13,0.020249,0.001578,0.000826,0.000945,0.000063,...,0.001042,0.000161,0.000257,0.046617,0.000156,0.000006,0.000537,0.000339,0.008437,0.000470
1,video4.webm,0.923,33,0.667,12,0.000118,0.000891,0.000188,0.000045,0.000063,...,0.000582,0.000393,0.000864,0.000947,0.000136,0.000007,0.000360,0.000159,0.001025,0.000020
2,video6.webm,0.863,33,0.700,10,0.011765,0.000746,0.000784,0.000013,0.000007,...,0.000224,0.000003,0.000031,0.002538,0.000104,0.000005,0.000064,0.005380,0.001027,0.001384
3,video8.webm,0.922,33,0.818,11,0.000223,0.000165,0.000007,0.000016,0.000005,...,0.000046,0.000009,0.000023,0.000053,0.000048,0.000019,0.000001,0.000004,0.000380,0.000029
4,video10.webm,0.950,34,0.900,10,0.000090,0.000615,0.003436,0.001281,0.003551,...,0.000037,0.000690,0.000171,0.000231,0.000637,0.000040,0.000061,0.000075,0.000002,0.001323


###splitting test and train data

In [0]:
x_c3d_short_train, x_c3d_short_test, y_c3d_short_train, y_c3d_short_test = train_test_split(dataset3.drop(['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations'], axis=1), dataset3['short-term_memorability'], test_size=0.2, random_state=101)
x_c3d_long_train, x_c3d_long_test, y_c3d_long_train, y_c3d_long_test = train_test_split(dataset3.drop(['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations'], axis=1), dataset3['long-term_memorability'], test_size=0.2, random_state=101)

###MODEL 1 : Linear Regression

In [0]:
from sklearn.linear_model import LinearRegression 

model_c3d_short_linearReg = LinearRegression ()
model_c3d_short_linearReg.fit(x_c3d_short_train, y_c3d_short_train)

output_c3d_short_linearReg=model_c3d_short_linearReg.predict(x_c3d_short_test)

print('Short Term Memorability')
Get_score(output_c3d_short_linearReg,y_c3d_short_test)





model_c3d_long_linearReg = LinearRegression ()
model_c3d_long_linearReg.fit(x_c3d_long_train, y_c3d_long_train)

output_c3d_long_linearReg=model_c3d_long_linearReg.predict(x_c3d_long_test)

print('Long Term Memorability')
Get_score(output_c3d_long_linearReg,y_c3d_long_test)


Short Term Memorability
The Spearman's correlation coefficient is: 0.251
Long Term Memorability
The Spearman's correlation coefficient is: 0.074


###MODEL 2 : Lasso Linear Regression

In [0]:
from sklearn.linear_model import LassoCV 

model_c3d_short_lasso = LassoCV (normalize=True,cv=5)
model_c3d_short_lasso.fit(x_c3d_short_train, y_c3d_short_train)

output_c3d_short_lasso=model_c3d_short_lasso.predict(x_c3d_short_test)

print('Short Term Memorability')
Get_score(output_c3d_short_lasso,y_c3d_short_test)




model_c3d_long_lasso = LassoCV (normalize=True,cv=5)
model_c3d_long_lasso.fit(x_c3d_long_train, y_c3d_long_train)

output_c3d_long_lasso=model_c3d_long_lasso.predict(x_c3d_long_test)

print('Long Term Memorability')
Get_score(output_c3d_long_lasso,y_c3d_long_test)


Short Term Memorability
The Spearman's correlation coefficient is: 0.259
Long Term Memorability
The Spearman's correlation coefficient is: 0.066


###MODEL3: Random Forest

In [0]:
from sklearn.ensemble import RandomForestRegressor 

model_c3d_short_randomForest = RandomForestRegressor (n_estimators =1500, max_features = 'sqrt')
model_c3d_short_randomForest.fit(x_c3d_short_train, y_c3d_short_train)

output_c3d_short_randomForest=model_c3d_short_randomForest.predict(x_c3d_short_test)

print('Short Term Memorability')
Get_score(output_c3d_short_randomForest,y_c3d_short_test)




model_c3d_long_randomForest = RandomForestRegressor (n_estimators =1500, max_features = 'sqrt')
model_c3d_long_randomForest.fit(x_c3d_long_train, y_c3d_long_train)

output_c3d_long_randomForest=model_c3d_long_randomForest.predict(x_c3d_long_test)

print('Long Term Memorability')
Get_score(output_c3d_long_randomForest,y_c3d_long_test)


Short Term Memorability
The Spearman's correlation coefficient is: 0.318
Long Term Memorability
The Spearman's correlation coefficient is: 0.113


### Checking to see if all Datasets are equal (ie random state split resulted into same Y_test)

In [51]:
print (y_c3d_short_test.equals(y_caption_short_test))
print (y_caption_short_test.equals(y_aesthetic_short_test))
print (y_aesthetic_short_test.equals(y_c3d_short_test))

True
True
True


#Ensemble Approach

SHORT TERM MEMORABILITY

**Caption :**

MODEL 1: Linear Regression : 0.356

MODEL 2: SVR : 0.303

MODEL 3: Bayesian Ridge : 0.417



**Aesthetic :**

MODEL 1 : Linear Regression : 0.237

MODEL 2 : Decision Tree Regressor : 0.150

MODEL 3 : Bayesian Ridge : 0.248



**C3D:**

MODEL 1 : Linear Regression : 0.251

MODEL 2 : Lasso Linear Regression : 0.259

MODEL 3 : Random Forest : 0.318

In [0]:
shortTermMemorability = (0.6 * output_caption_short_bayesian) + (0.15 * output_aesthetic_short_bayesian) + (0.25 * output_c3d_short_randomForest)

In [44]:
Get_score(shortTermMemorability,y_c3d_short_test)

The Spearman's correlation coefficient is: 0.439


LONG TERM MEMORABILITY

**Caption : **

MODEL 1: Linear Regression : 0.130

MODEL 2: SVR : 0.112

MODEL 3: Bayesian Ridge : 0.154


**Aesthetic :**

MODEL 1 : Linear Regression : 0.088

MODEL 2 : Decision Tree Regressor : 0.028

MODEL 3 : Bayesian Ridge : 0.062


**C3D:**

MODEL 1 : Linear Regression : 0.074

MODEL 2 : Lasso Linear Regression : 0.066

MODEL 3 : Random Forest : 0.113

In [0]:
longTermMemorability = (0.4 * output_caption_long_bayesian) + (0.2 * output_aesthetic_long_linearReg) + (0.4 * output_c3d_long_randomForest)

In [54]:
Get_score(longTermMemorability,y_c3d_long_test)

The Spearman's correlation coefficient is: 0.174


# Creating Final Model for predictions  


###Loading Test captions 

In [0]:
test_captions_path = '/content/drive/My Drive/CA684_Assignment_Data/test-set/test-set-1_video-captions.txt'
test_captions=read_caps(test_captions_path)

In [271]:
test_captions.head(5)

,video,caption
0,video7494.webm,green-jeep-struggling-to-drive-over-huge-rocks
1,video7495.webm,hiking-woman-tourist-is-walking-forward-in-mou...
2,video7496.webm,close-up-of-african-american-doctors-hands-usi...
3,video7497.webm,slow-motion-of-a-man-using-treadmill-in-the-gy...
4,video7498.webm,slow-motion-of-photographer-in-national-park


In [0]:
test_ground_truth_path = '/content/drive/My Drive/CA684_Assignment_Data/test-set/ground_truth_template.csv'
test_groundTruth=pd.read_csv(test_ground_truth_path)

In [273]:
test_groundTruth.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,7494,NaN,33,NaN,12
1,7495,NaN,34,NaN,10
2,7496,NaN,32,NaN,13
3,7497,NaN,33,NaN,10
4,7498,NaN,33,NaN,10


In [0]:
test_groundTruth['video'] = 'video'+test_groundTruth['video'].astype(str)+'.webm'

In [275]:
test_groundTruth.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video7494.webm,NaN,33,NaN,12
1,video7495.webm,NaN,34,NaN,10
2,video7496.webm,NaN,32,NaN,13
3,video7497.webm,NaN,33,NaN,10
4,video7498.webm,NaN,33,NaN,10


In [0]:
test_dataset1 = pd.merge(test_groundTruth,test_captions, on ="video")

In [277]:
test_dataset1.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,caption
0,video7494.webm,NaN,33,NaN,12,green-jeep-struggling-to-drive-over-huge-rocks
1,video7495.webm,NaN,34,NaN,10,hiking-woman-tourist-is-walking-forward-in-mou...
2,video7496.webm,NaN,32,NaN,13,close-up-of-african-american-doctors-hands-usi...
3,video7497.webm,NaN,33,NaN,10,slow-motion-of-a-man-using-treadmill-in-the-gy...
4,video7498.webm,NaN,33,NaN,10,slow-motion-of-photographer-in-national-park


###Data Cleaning of review column in Test Dataset

In [278]:
import nltk
import re
import string
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words('english'))
snow = nltk.stem.LancasterStemmer('english')
ps = PorterStemmer() 
tokenizer = RegexpTokenizer(r'\w+')

corpus = [] 

for sentence in test_dataset1['caption'] :
  # column : "Review", row ith 
  review = re.sub('[^a-zA-Z]', ' ', sentence) 
  
  # convert all cases to lower cases 
  review = review.lower()  
  
  # split to array(default delimiter is " ") 
  review = tokenizer.tokenize(review)
 
  
  review = [ps.stem(word) for word in review if not word in stop_words] 
  
  # rejoin all string array elements 
    # to create back into a string 
  review = ' '.join(review)
  
  
 # append each string to create 
    # array of clean text  
  corpus.append(review) 
  
  
test_dataset1['corpus']= corpus
test_dataset1.head(5)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,caption,corpus
0,video7494.webm,NaN,33,NaN,12,green-jeep-struggling-to-drive-over-huge-rocks,green jeep struggl drive huge rock
1,video7495.webm,NaN,34,NaN,10,hiking-woman-tourist-is-walking-forward-in-mou...,hike woman tourist walk forward mountain sunse...
2,video7496.webm,NaN,32,NaN,13,close-up-of-african-american-doctors-hands-usi...,close african american doctor hand use sphygmo...
3,video7497.webm,NaN,33,NaN,10,slow-motion-of-a-man-using-treadmill-in-the-gy...,slow motion man use treadmil gym regular physi...
4,video7498.webm,NaN,33,NaN,10,slow-motion-of-photographer-in-national-park,slow motion photograph nation park


###Using TfidfVectorizer and ngram(unigram & bigram) to convert string to numeric values to feed to the model

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizerfinal = TfidfVectorizer(max_features=1500, ngram_range=(1,2) )
vectorizerfinal.fit(dataset1['corpus'])

final_caption_train=vectorizerfinal.transform(dataset1['corpus'])
final_caption_test=vectorizerfinal.transform(test_dataset1['corpus'])

##MODEL 1 : Caption
###Short Term - Bayesian Ridge
###Long Term - Bayesian Ridge



In [280]:
from sklearn.linear_model import BayesianRidge

final_model_caption_short_bayesian = BayesianRidge ()
final_model_caption_short_bayesian.fit(final_caption_train.toarray(), dataset1['short-term_memorability'])

final_output_caption_short_bayesian=final_model_caption_short_bayesian.predict(final_caption_test.toarray())

final_output_caption_short_bayesian[1:5]






from sklearn.linear_model import BayesianRidge

final_model_caption_long_bayesian = BayesianRidge ()
final_model_caption_long_bayesian.fit(final_caption_train.toarray(), dataset1['long-term_memorability'])

final_output_caption_long_bayesian=final_model_caption_long_bayesian.predict(final_caption_test.toarray())

final_output_caption_long_bayesian[1:5]


array([0.7404419 , 0.83501417, 0.78076236, 0.73316659])

In [0]:
output_dataframe = test_dataset1

In [0]:
output_dataframe['caption_short_output'] = final_output_caption_short_bayesian
output_dataframe['caption_long_output'] = final_output_caption_long_bayesian


In [283]:
output_dataframe.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,caption,corpus,caption_short_output,caption_long_output
0,video7494.webm,NaN,33,NaN,12,green-jeep-struggling-to-drive-over-huge-rocks,green jeep struggl drive huge rock,0.848973,0.748902
1,video7495.webm,NaN,34,NaN,10,hiking-woman-tourist-is-walking-forward-in-mou...,hike woman tourist walk forward mountain sunse...,0.764251,0.740442
2,video7496.webm,NaN,32,NaN,13,close-up-of-african-american-doctors-hands-usi...,close african american doctor hand use sphygmo...,0.921353,0.835014
3,video7497.webm,NaN,33,NaN,10,slow-motion-of-a-man-using-treadmill-in-the-gy...,slow motion man use treadmil gym regular physi...,0.885972,0.780762
4,video7498.webm,NaN,33,NaN,10,slow-motion-of-photographer-in-national-park,slow motion photograph nation park,0.844159,0.733167


##MODEL 2 : Aesthetic
###Short Term - Bayesian Ridge
###Long Term - Linear regression

###load test aesthetic features

In [0]:
path="/content/drive/My Drive/CA684_Assignment_Data/test-set/aesthetic_vis_features_test/median/"
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
   for file in f:
       if '.txt' in file:
           files.append(os.path.join(r, file))

In [0]:
def read_features_from_file(dataframe, filename):
  dataframe_1 = pd.read_csv(filename, header=None)
  dataframe_1['video'] = filename[filename.find('video'):filename.find('.txt')] + '.webm'
  dataframe = dataframe.append(dataframe_1, ignore_index=True)
  return dataframe

In [0]:
dataframe_aesthetic_test = pd.DataFrame()
for filename in files:
  dataframe_aesthetic_test = read_features_from_file(dataframe_aesthetic_test, filename)

In [287]:
dataframe_aesthetic_test.head(5)

,0,1,2,3,4,5,6,7,8,9,...,100,101,102,103,104,105,106,107,108,video
0,0.461159,0.180321,0.609216,0.192552,0.573822,0.299892,48.416986,0.575000,0.021221,3.0,...,-0.000000,0.842840,0.456026,0.381484,1.698315,0.0,0.165206,0.322674,0.415184,video8759.webm
1,0.547819,0.299441,0.832418,0.574203,0.706239,0.548596,53.938527,0.584967,0.015614,1.0,...,-0.000001,0.833881,0.250386,0.297984,1.250002,0.0,0.013718,0.229135,0.027516,video8762.webm
2,0.276918,0.121264,0.649454,0.136974,0.612684,0.338838,51.048896,0.135802,0.006227,2.0,...,0.000003,0.740355,0.338121,0.234920,1.177828,0.0,0.378273,0.315327,0.393496,video8758.webm
3,0.195197,0.341496,0.413990,0.237250,0.339475,0.360236,42.808806,0.592409,0.023402,3.0,...,0.000324,0.000000,0.244494,0.094539,0.948351,0.0,0.154096,-0.560893,0.370832,video8768.webm
4,0.411114,0.237132,0.528279,0.207872,0.481215,0.205192,46.094116,0.106061,0.044607,6.0,...,0.000014,0.039444,0.413060,0.539532,1.561741,0.0,0.616250,0.480776,0.240629,video8770.webm


In [0]:
dataframe_aesthetic_test.to_csv('test_aesthetic.csv')

In [0]:
test_aesthetic_path = '/content/drive/My Drive/test_aesthetic.csv'
test_aesthetic=pd.read_csv(aesthetic_path,index_col=0)

In [290]:
test_dataset2 = pd.merge(test_groundTruth,dataframe_aesthetic_test, on ="video")
test_dataset2.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,0,1,2,3,4,...,99,100,101,102,103,104,105,106,107,108
0,video7494.webm,NaN,33,NaN,12,0.151128,0.428798,0.388939,0.319523,0.319127,...,-0.002096,0.000004,0.000000,0.181365,0.260591,1.343774,0.0,-0.051514,-0.312289,-0.037960
1,video7495.webm,NaN,34,NaN,10,0.351501,0.387280,0.524060,0.410809,0.429068,...,-0.000263,0.000056,0.292160,0.451259,0.296964,1.298557,0.0,0.587843,0.664178,1.331725
2,video7496.webm,NaN,32,NaN,13,0.126592,0.225055,0.479929,0.172715,0.432944,...,0.004709,0.000142,0.013248,0.079113,0.322459,1.115463,0.0,0.042739,0.680311,0.905298
3,video7497.webm,NaN,33,NaN,10,0.218887,0.337801,0.633643,0.245733,0.529765,...,0.000135,0.000000,0.000000,0.497551,0.308292,2.034225,0.0,0.557117,-0.055591,1.013607
4,video7498.webm,NaN,33,NaN,10,0.438498,0.290777,0.474659,0.267142,0.403033,...,-0.000766,0.000066,0.453850,0.470586,0.266092,1.642591,0.0,0.037238,0.416894,0.419495


In [0]:
from sklearn.linear_model import BayesianRidge

final_model_aesthetic_short_bayesian= BayesianRidge ()
final_model_aesthetic_short_bayesian.fit(dataset2.drop(['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations'], axis=1), dataset2['short-term_memorability'])

final_output_aesthetic_short_bayesian=final_model_aesthetic_short_bayesian.predict(test_dataset2.drop(['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations'], axis=1))




from sklearn.linear_model import LinearRegression 
final_model_aesthetic_long_linearReg= LinearRegression ()
final_model_aesthetic_long_linearReg.fit(dataset2.drop(['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations'], axis=1), dataset2['long-term_memorability'])

final_output_aesthetic_long_linearReg=final_model_aesthetic_long_linearReg.predict(test_dataset2.drop(['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations'], axis=1))


In [292]:
output_dataframe['aesthetic_short_output'] = final_output_aesthetic_short_bayesian
output_dataframe['aesthetic_long_output'] = final_output_aesthetic_long_linearReg

output_dataframe.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,caption,corpus,caption_short_output,caption_long_output,aesthetic_short_output,aesthetic_long_output
0,video7494.webm,NaN,33,NaN,12,green-jeep-struggling-to-drive-over-huge-rocks,green jeep struggl drive huge rock,0.848973,0.748902,0.851031,0.749921
1,video7495.webm,NaN,34,NaN,10,hiking-woman-tourist-is-walking-forward-in-mou...,hike woman tourist walk forward mountain sunse...,0.764251,0.740442,0.838789,0.742697
2,video7496.webm,NaN,32,NaN,13,close-up-of-african-american-doctors-hands-usi...,close african american doctor hand use sphygmo...,0.921353,0.835014,0.865397,0.798661
3,video7497.webm,NaN,33,NaN,10,slow-motion-of-a-man-using-treadmill-in-the-gy...,slow motion man use treadmil gym regular physi...,0.885972,0.780762,0.869349,0.795936
4,video7498.webm,NaN,33,NaN,10,slow-motion-of-photographer-in-national-park,slow motion photograph nation park,0.844159,0.733167,0.839037,0.791368


## Model 3 : C3D 
###Short Term - Random Forest
###Long Term - Random Forest

In [0]:
path="/content/drive/My Drive/CA684_Assignment_Data/test-set/C3D_test"
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
   for file in f:
       if '.txt' in file:
           files.append(os.path.join(r, file))

In [0]:
def read_features_from_file_c3d(dataframe, filename):
  dataframe_1 = pd.read_csv(filename, header=None, sep=' ')
  dataframe_1['video'] = filename[filename.find('video'):filename.find('.txt')] + '.webm'
  dataframe = dataframe.append(dataframe_1, ignore_index=True)
  return dataframe

In [0]:
dataframe_c3d_test = pd.DataFrame()
for filename in files:
  dataframe_c3d_test = read_features_from_file_c3d(dataframe_c3d_test, filename)

In [296]:
dataframe_c3d_test.head(5)

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,video
0,0.003526,0.001376,0.046187,0.000008,0.000041,0.000295,0.000343,0.000406,0.000070,0.000015,...,1.673800e-04,5.811330e-03,0.000252,4.400000e-07,0.000568,0.000051,0.000067,0.682813,NaN,video8768.webm
1,0.007772,0.002302,0.003671,0.000048,0.000017,0.000018,0.000262,0.000129,0.000079,0.000027,...,1.110000e-05,8.693400e-04,0.000065,6.150000e-06,0.000271,0.017517,0.666933,0.002858,NaN,video8759.webm
2,0.001573,0.001398,0.071723,0.000025,0.000019,0.000827,0.006838,0.000136,0.000029,0.000239,...,1.279700e-04,1.019300e-04,0.000007,3.577000e-05,0.001544,0.000034,0.000837,0.804485,NaN,video8762.webm
3,0.000358,0.003973,0.008803,0.007741,0.000403,0.000194,0.009095,0.017529,0.000285,0.000026,...,2.662220e-03,2.824549e-02,0.000521,5.476000e-05,0.002431,0.000706,0.045552,0.001591,NaN,video8765.webm
4,0.000001,0.000012,0.000025,0.000012,0.000058,0.000572,0.000002,0.000002,0.000030,0.000003,...,2.300000e-07,4.500000e-07,0.000002,3.964600e-04,0.000020,0.000001,0.000026,0.000154,NaN,video8758.webm


In [0]:
dataframe_c3d_test.to_csv('test_c3d.csv')

In [0]:
test_c3d_path = '/content/drive/My Drive/test_c3d.csv'
test_c3d=pd.read_csv(test_c3d_path,index_col=0)

In [0]:
test_dataset3 = pd.merge(test_groundTruth,dataframe_c3d_test, on ="video")


In [300]:
test_dataset3.drop(columns=101,inplace=True)
test_dataset3.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,0,1,2,3,4,...,91,92,93,94,95,96,97,98,99,100
0,video7494.webm,NaN,33,NaN,12,0.000136,0.000213,0.001248,0.000011,1.180000e-06,...,0.000003,5.620000e-06,1.260000e-06,0.000199,1.500000e-06,2.000000e-07,5.840000e-06,0.000006,0.000210,0.195120
1,video7495.webm,NaN,34,NaN,10,0.000127,0.000024,0.049501,0.000024,2.361000e-05,...,0.000173,3.019200e-04,2.649800e-04,0.000163,1.050000e-05,2.540000e-06,3.619300e-04,0.000016,0.000010,0.000392
2,video7496.webm,NaN,32,NaN,13,0.488982,0.000585,0.000025,0.000160,2.400000e-07,...,0.000002,1.900000e-07,3.200000e-07,0.001802,2.640000e-06,9.000000e-08,2.600000e-07,0.000067,0.000123,0.000015
3,video7497.webm,NaN,33,NaN,10,0.249251,0.001018,0.000008,0.000019,2.200000e-07,...,0.000065,8.400000e-07,1.700000e-07,0.000808,9.700000e-07,7.670000e-06,9.100000e-07,0.000015,0.000707,0.000010
4,video7498.webm,NaN,33,NaN,10,0.009894,0.000034,0.002613,0.000002,1.010000e-06,...,0.000012,6.600000e-07,8.770000e-06,0.000132,7.400000e-07,2.120000e-06,8.260000e-06,0.001402,0.000745,0.000456


In [301]:
dataset3.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,0,1,2,3,4,...,91,92,93,94,95,96,97,98,99,100
0,video3.webm,0.924,34,0.846,13,0.020249,0.001578,0.000826,0.000945,0.000063,...,0.001042,0.000161,0.000257,0.046617,0.000156,0.000006,0.000537,0.000339,0.008437,0.000470
1,video4.webm,0.923,33,0.667,12,0.000118,0.000891,0.000188,0.000045,0.000063,...,0.000582,0.000393,0.000864,0.000947,0.000136,0.000007,0.000360,0.000159,0.001025,0.000020
2,video6.webm,0.863,33,0.700,10,0.011765,0.000746,0.000784,0.000013,0.000007,...,0.000224,0.000003,0.000031,0.002538,0.000104,0.000005,0.000064,0.005380,0.001027,0.001384
3,video8.webm,0.922,33,0.818,11,0.000223,0.000165,0.000007,0.000016,0.000005,...,0.000046,0.000009,0.000023,0.000053,0.000048,0.000019,0.000001,0.000004,0.000380,0.000029
4,video10.webm,0.950,34,0.900,10,0.000090,0.000615,0.003436,0.001281,0.003551,...,0.000037,0.000690,0.000171,0.000231,0.000637,0.000040,0.000061,0.000075,0.000002,0.001323


In [0]:
from sklearn.ensemble import RandomForestRegressor 

final_model_c3d_short_randomForest= RandomForestRegressor (n_estimators =1500, max_features = 'sqrt')
final_model_c3d_short_randomForest.fit(dataset3.drop(['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations'], axis=1), dataset3['short-term_memorability'])

final_output_c3d_short_randomForest=final_model_c3d_short_randomForest.predict(test_dataset3.drop(['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations'], axis=1))




from sklearn.ensemble import RandomForestRegressor 
final_model_c3d_long_randomForest= RandomForestRegressor (n_estimators =1500, max_features = 'sqrt')
final_model_c3d_long_randomForest.fit(dataset3.drop(['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations'], axis=1), dataset3['long-term_memorability'])

final_output_c3d_long_randomForest=final_model_c3d_long_randomForest.predict(test_dataset3.drop(['video','short-term_memorability','nb_short-term_annotations','long-term_memorability','nb_long-term_annotations'], axis=1))


In [303]:
output_dataframe['c3d_short_output'] = final_output_c3d_short_randomForest
output_dataframe['c3d_long_output'] = final_output_c3d_long_randomForest

output_dataframe.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,caption,corpus,caption_short_output,caption_long_output,aesthetic_short_output,aesthetic_long_output,c3d_short_output,c3d_long_output
0,video7494.webm,NaN,33,NaN,12,green-jeep-struggling-to-drive-over-huge-rocks,green jeep struggl drive huge rock,0.848973,0.748902,0.851031,0.749921,0.864075,0.788367
1,video7495.webm,NaN,34,NaN,10,hiking-woman-tourist-is-walking-forward-in-mou...,hike woman tourist walk forward mountain sunse...,0.764251,0.740442,0.838789,0.742697,0.830033,0.728149
2,video7496.webm,NaN,32,NaN,13,close-up-of-african-american-doctors-hands-usi...,close african american doctor hand use sphygmo...,0.921353,0.835014,0.865397,0.798661,0.887062,0.806645
3,video7497.webm,NaN,33,NaN,10,slow-motion-of-a-man-using-treadmill-in-the-gy...,slow motion man use treadmil gym regular physi...,0.885972,0.780762,0.869349,0.795936,0.883276,0.788033
4,video7498.webm,NaN,33,NaN,10,slow-motion-of-photographer-in-national-park,slow motion photograph nation park,0.844159,0.733167,0.839037,0.791368,0.855857,0.775769


#Final Predictions

In [0]:
output_dataframe['short-term_memorability'] = (0.6 * output_dataframe['caption_short_output']) + (0.15 * output_dataframe['aesthetic_short_output']) + (0.25 * output_dataframe['c3d_short_output'])

In [0]:
output_dataframe['long-term_memorability'] = (0.4 * output_dataframe['caption_long_output']) + (0.2 * output_dataframe['aesthetic_long_output']) + (0.4 * output_dataframe['c3d_long_output'])

In [306]:
output_dataframe.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,caption,corpus,caption_short_output,caption_long_output,aesthetic_short_output,aesthetic_long_output,c3d_short_output,c3d_long_output
0,video7494.webm,0.853058,33,0.764892,12,green-jeep-struggling-to-drive-over-huge-rocks,green jeep struggl drive huge rock,0.848973,0.748902,0.851031,0.749921,0.864075,0.788367
1,video7495.webm,0.791877,34,0.735976,10,hiking-woman-tourist-is-walking-forward-in-mou...,hike woman tourist walk forward mountain sunse...,0.764251,0.740442,0.838789,0.742697,0.830033,0.728149
2,video7496.webm,0.904387,32,0.816396,13,close-up-of-african-american-doctors-hands-usi...,close african american doctor hand use sphygmo...,0.921353,0.835014,0.865397,0.798661,0.887062,0.806645
3,video7497.webm,0.882804,33,0.786705,10,slow-motion-of-a-man-using-treadmill-in-the-gy...,slow motion man use treadmil gym regular physi...,0.885972,0.780762,0.869349,0.795936,0.883276,0.788033
4,video7498.webm,0.846315,33,0.761848,10,slow-motion-of-photographer-in-national-park,slow motion photograph nation park,0.844159,0.733167,0.839037,0.791368,0.855857,0.775769


###Creating final dataframe to export 

In [0]:
def change_video_name_remove(val):
 val = str(val)[str(val).find('o')+1:str(val).find('.')]
 return val

In [0]:
resultset=output_dataframe

In [309]:
resultset.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,caption,corpus,caption_short_output,caption_long_output,aesthetic_short_output,aesthetic_long_output,c3d_short_output,c3d_long_output
0,video7494.webm,0.853058,33,0.764892,12,green-jeep-struggling-to-drive-over-huge-rocks,green jeep struggl drive huge rock,0.848973,0.748902,0.851031,0.749921,0.864075,0.788367
1,video7495.webm,0.791877,34,0.735976,10,hiking-woman-tourist-is-walking-forward-in-mou...,hike woman tourist walk forward mountain sunse...,0.764251,0.740442,0.838789,0.742697,0.830033,0.728149
2,video7496.webm,0.904387,32,0.816396,13,close-up-of-african-american-doctors-hands-usi...,close african american doctor hand use sphygmo...,0.921353,0.835014,0.865397,0.798661,0.887062,0.806645
3,video7497.webm,0.882804,33,0.786705,10,slow-motion-of-a-man-using-treadmill-in-the-gy...,slow motion man use treadmil gym regular physi...,0.885972,0.780762,0.869349,0.795936,0.883276,0.788033
4,video7498.webm,0.846315,33,0.761848,10,slow-motion-of-photographer-in-national-park,slow motion photograph nation park,0.844159,0.733167,0.839037,0.791368,0.855857,0.775769


In [0]:
resultset['video'] = resultset['video'].apply(change_video_name_remove)

In [311]:
resultset.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,caption,corpus,caption_short_output,caption_long_output,aesthetic_short_output,aesthetic_long_output,c3d_short_output,c3d_long_output
0,7494,0.853058,33,0.764892,12,green-jeep-struggling-to-drive-over-huge-rocks,green jeep struggl drive huge rock,0.848973,0.748902,0.851031,0.749921,0.864075,0.788367
1,7495,0.791877,34,0.735976,10,hiking-woman-tourist-is-walking-forward-in-mou...,hike woman tourist walk forward mountain sunse...,0.764251,0.740442,0.838789,0.742697,0.830033,0.728149
2,7496,0.904387,32,0.816396,13,close-up-of-african-american-doctors-hands-usi...,close african american doctor hand use sphygmo...,0.921353,0.835014,0.865397,0.798661,0.887062,0.806645
3,7497,0.882804,33,0.786705,10,slow-motion-of-a-man-using-treadmill-in-the-gy...,slow motion man use treadmil gym regular physi...,0.885972,0.780762,0.869349,0.795936,0.883276,0.788033
4,7498,0.846315,33,0.761848,10,slow-motion-of-photographer-in-national-park,slow motion photograph nation park,0.844159,0.733167,0.839037,0.791368,0.855857,0.775769


In [0]:
resultset.drop(columns=['caption','corpus','caption_short_output','caption_long_output','aesthetic_short_output','aesthetic_long_output','c3d_short_output','c3d_long_output'],inplace=True)

In [0]:
resultset.to_csv('memorability_scores.csv',index=None)